# 数値型

In [1]:
import sys
import numpy as np
import pandas as pd
sys.path.append("../src")
from data_loader import load_hotel_reserve, load_production, load_production_missing_num

customer_tb, hotel_tb, reserve_tb = load_hotel_reserve()
production_tb = load_production()
production_miss_num = load_production_missing_num()

## 数値型への変換

In [2]:
type(40000 / 3)

float

In [3]:
# 整数型へ変換
print(int(40000 / 3))

# 浮動小数点型へ変換
print(float(40000 / 3))


13333
13333.333333333334


In [4]:
df = pd.DataFrame({'value': [40000 / 3]})
# データ型の確認
df.dtypes

value    float64
dtype: object

In [5]:
# 整数型へ変換
df['value'].astype('int8')
df['value'].astype('int16')
df['value'].astype('int32')
df['value'].astype('int64')

# 浮動小数点型へ変換
df['value'].astype('float16')
df['value'].astype('float32')
df['value'].astype('float64')
df['value'].astype('float128')

# 下記のようにpythonのデータ型を指定できる
df['value'].astype(int)
df['value'].astype(float)

0    13333.333333
Name: value, dtype: float64

## 対数化による非線形な変換
予約テーブルのtotal_priceを1000で割ってから、底10で対数化

In [6]:
reserve_tb['total_price_log'] = \
    reserve_tb['total_price'].apply(lambda x: np.log(x / 1000 + 1))


## カテゴリ化による非線形な変換
数値型をカテゴリ化することで非線形な関係を表現することができる。  
顧客テーブルを年齢を10きざみのカテゴリ型として追加する。

In [7]:
customer_tb['age_rank'] = \
    (np.floor(customer_tb['age'] / 10) * 10).astype('category')


In [8]:
customer_tb.tail()

,customer_id,age,sex,home_latitude,home_longitude,age_rank
995,c_996,44,man,34.465648,135.373787,40.0
996,c_997,35,man,35.345372,139.413754,30.0
997,c_998,32,woman,43.062267,141.272126,30.0
998,c_999,48,woman,38.172800,140.464198,40.0
999,c_1000,39,man,35.452412,139.411310,30.0


## 正規化
1. 平均0、分散1に変換する正規化
2. 最小値0、最大値1に変換する正規化

In [9]:
from sklearn.preprocessing import StandardScaler

# 正規化を行うオブジェクトを生成
ss = StandardScaler()

# fit_transform関数は、fit関数（正規化するための前準備の計算）と
# transform関数（準備された情報から正規化の変換処理を行う）の両方を行う
result = ss.fit_transform(reserve_tb[['people_num', 'total_price']])

reserve_tb['people_num_normalized'] = [x[0] for x in result]
reserve_tb['total_price_normalized'] = [x[1] for x in result]

In [10]:
reserve_tb.head()

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,total_price_log,people_num_normalized,total_price_normalized
0,r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200,4.587006,1.300709,-0.053194
1,r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600,3.072693,-0.483753,-0.747822
2,r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600,3.543854,-0.483753,-0.629935
3,r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400,5.275049,1.300709,0.828240
4,r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100,4.235555,0.408478,-0.317080


## 外れ値の除去
極端に大きな値や小さな値は予測モデルを構築するときに悪影響を与えることが多く、前処理で除去することが求められる。  
正規分布を前提にした、最も簡単でよく扱われる外れ値検出の方法は、平均値から標準偏差の一定値倍以上離れた値を除外すること。一般に一定倍数は3より大きな値を設定する。  
予約テーブルの予約合計金額(total_price)において、平均値から標準偏差の3倍以内の値に収まる予約レコードのみに絞る。

In [11]:
reserve_tb = reserve_tb[
    (abs(reserve_tb['total_price'] - np.mean(reserve_tb['total_price'])) /
     np.std(reserve_tb['total_price']) <= 3)
].reset_index()


## 主成分分析による次元圧縮
要素間の相関を排除し、できるだけ少ない情報の損失で、新たな要素を定義する。モデルの精度が大きく上がることはないが、データの可視化を容易にすることができたり、新たな次元から新たな発見を見つけることができる。

In [12]:
from sklearn.decomposition import PCA

# n_componentsに、主成分分析で変換後の次元数を設定
pca = PCA(n_components=2)

# 主成分分析を実行
# pcaに主成分分析の変換パラメータが保存され、返り値に主成分分析後の値が返される
pca_values = pca.fit_transform(production_tb[['length', 'thickness']])

# 累積寄与率と寄与率の確認
print('累積寄与率: {0}'.format(sum(pca.explained_variance_ratio_)))
print('各次元の寄与率: {0}'.format(pca.explained_variance_ratio_))

# predict関数を利用し、同じ次元圧縮処理を実行
pca_newvalues = pca.transform(production_tb[['length', 'thickness']])

累積寄与率: 1.0
各次元の寄与率: [0.97897794 0.02102206]


## 数値の補完
欠損値の前処理を行う前に、どのような欠損なのかを確認する必要がある。
- MCAR (Missing Completely At Random)：偶然に起きている完全なランダムな欠損。例えば、室温を測る温度センサーから送られてくるデータが一定の確率で破損する場合。
- MAR (Missing At Random)：欠損した項目データとは関係なく、他の項目データに依存した欠損。例えば、室温を測る温度センサーから送られてくるデータが、湿度が高いほど破損する確率が高くなる場合。
- MNAR (Missing Not At Random)：欠損した項目データに依存した欠損。例えば、室温を測る温度センサーから送られてくるデータが40度を超えると破損する場合。

データを補完する方法にはさまざまな方法がある。
1. 定数による補完
2. 集計値による補完
3. 欠損していないデータに基づく予測値によって補完
4. 時系列の関係から補完
- 欠損しているデータの前後のデータから欠損値を予測して補完する方法
- 例えば、10:01の温度データが欠けているときに、10:00の温度データと10:02の温度データの平均値を利用して補完する
- 時間に対して連続している値が対称であれば、MCAR、MARにおいて有効
5. 多重代入法
- 特定の値を欠損部分に代入することでMCAR以外の場合にはバイアスが生じてしまう
- 補完したデータセットを複数作成し、そのそれぞれのデータセットに対して解析を行う
- 複数の結果を統合することでバイアスの少ない結果を得る
- MCAR、MARにおいて有効
6. 最尤法
- 多重代入法と同様に、本来のデータのばらつきよりも補完後のデータのばらつきが小さくする手法
- 潜在変数を導入し、EMアルゴリズムを用いて尤度を最大化することで欠損値を推定する
- MCAR、MARにおいて有効

MCAR、MARにおいては、多重代入法または最尤法を利用することが一般的

### thicknessが欠損しているレコードを削除する

In [13]:
# replace関数によって、Noneをnanに変換
# （Noneを指定する際には文字列として指定する必要がある）
production_miss_num.replace('None', np.nan, inplace=True)

# dropna関数によって、thicknessにnanを含むレコードを削除
production_miss_num.dropna(subset=['thickness'], inplace=True)

### 定数補完
欠損しているthicknessを1の値で補完する

In [14]:
production_miss_num = load_production_missing_num()

In [15]:
# replace関数によって、Noneをnanに変換
production_miss_num.replace('None', np.nan, inplace=True)

# fillna関数によって、thicknessの欠損値を1で補完
production_miss_num['thickness'].fillna(1, inplace=True)

### 平均値補完

In [16]:
production_miss_num = load_production_missing_num()

In [17]:
# replace関数によって、Noneをnanに変換
production_miss_num.replace('None', np.nan, inplace=True)

# thicknessを数値型に変換（Noneが混ざっているため数値型になっていない）
production_miss_num['thickness'] = \
    production_miss_num['thickness'].astype('float64')

# thicknessの平均値を計算
thickness_mean = production_miss_num['thickness'].mean()

# thicknessの欠損値をthicknessの平均値で補完
production_miss_num['thickness'].fillna(thickness_mean, inplace=True)


### PMMによる多重代入法
**PMM** (Predictive Mean Matching)
1. 欠損データを除いたデータから欠損データを予測する回帰モデルを構築
2. 構築した回帰モデルの係数と誤差分散の分布を計算
3. 係数と誤差分散の分布から新たな係数と誤差分散の値を生成
4. 3で生成した係数と誤差分散の値にしたがった回帰モデルから予測値を計算
5. 欠損していない観測データの中から予測値に最も近いデータを補完値として採用
6. データを補完して、新たな構築した回帰モデルの係数と誤差分散の分布を計算、3に戻る

3-6を補完する値の分布が安定するまで繰り返し、安定してから指定したデータセットの数分の補完値が得られたら終了となる。

In [18]:
production_miss_num = load_production_missing_num()

In [19]:
from fancyimpute import MICE

# replace関数によって、Noneをnanに変換
production_miss_num.replace('None', np.nan, inplace=True)

# mice関数を利用するためにデータ型を変換（mice関数内でモデル構築をするため）
production_miss_num['thickness'] = \
    production_miss_num['thickness'].astype('float64')
production_miss_num['type'] = \
    production_miss_num['type'].astype('category')
production_miss_num['fault_flg'] = \
    production_miss_num['fault_flg'].astype('category')

# ダミー変数化（「第9章 カテゴリ型」で詳しく解説)
production_dummy_flg = pd.get_dummies(
    production_miss_num[['type', 'fault_flg']], drop_first=True)

# mice関数にPMMを指定して、多重代入法を実施
# n_imputationsは取得するデータセットの数
# n_burn_inは値を取得する前に試行する回数
mice = MICE(n_imputations=10, n_burn_in=50, impute_type='pmm')

# 処理内部でTensorFlowを利用
production_mice = mice.multiple_imputations(
    # 数値の列とダミー変数を連結
    pd.concat([production_miss_num[['length', 'thickness']],
               production_dummy_flg], axis=1)
)

# 下記に補完する値が格納されている
production_mice[0]


ImportError: cannot import name 'MICE' from 'fancyimpute' (/home/aoyg/miniconda3/envs/myenv/lib/python3.8/site-packages/fancyimpute/__init__.py)

エラーにより実行できない